# NORTH Databehandling

Dette script lader dig læse og behandle data fra tokamakken NORTH. Scriptet er skrevet i programmeringssproget Python.

Koden er her delt op i forskellige små bider, såkaldte kodeblokkke, med kommentarer undervejs (markeret med et #-tegn). Kommentarer bruges kun til at forklare og forstå koden, og du kan derfor ændre kommentarer uden påvirkning på selve koden.


## Importer biblioteker

Første trin er at indlæse nogle såkaldte biblioteker, som vi vil benytte til at gøre databehandlingen nemmere.

In [ ]:
# Indlæs biblioteker og filstier

!git clone https://github.com/thrysoe/north.git -q
!pip install nptdms -q

import nptdms
from matplotlib import pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
from north.Diagnostics import Probe

data_path   = './north/Data/'
figure_path = './north/Figures/'

## Indlæs data

Nu vil vi indlæse forsøgsdata.

Først skal du uploade dine datafiler til Google Colab.

1. Kilk på mappeikonet ude til venstre.
2. Klik på mappen 'north' og derefter på undermappen 'Data'. Kan du ikke se disse mapper, skal du genindlæse siden (tryk på F5) og køre ovenstående kodeblok igen.
3. Højreklik på 'Data'-mappen, og vælg 'Upload'. DU SKAL UPLOADE 2 FILER (.tdms) FOR HVERT EKSPERIMENT, ELLERS VIRKER KODEN IKKE.

Desværre skal du nok igennem denne proces, hver gang du åbner Google Colab, fordi det ikke gemmes lokalt på din computer.

Du skal nu angive nummeret på det eksperiment du ønsker at undersøge. Vi kalder hvert eksperiment for et **shot**.

In [ ]:
# Nummeret på dit eksperiment på NORTH (skal matche nummeret på de filer, du har uploadet)
shot = 9774

Vi kan nu indlæse alle probedata.

In [ ]:
# Indlæs datafilen, hvori probedata er gemt
probe = {}
for i in range(Probe.TOTAL_PROBES):
    probe[i] = Probe(path = data_path, shot = shot, number = i+1, caching = True)

Proberne er placeret forksellige steder i NORTH. Vi kan tegne en oversigt over probepositionerne i et såkaldt poloidalt tværsnit i NORTH.

In [ ]:
# Påbegynd en figur
fig, ax = plt.subplots()

# Tegn NORTH's vakuumkammer (hvor plasmaet befinder sig)
theta = np.linspace(0, 2 * np.pi, 500)
circle_x = 250 + 130 * np.cos(theta)
circle_z = 130 * np.sin(theta)
ax.plot(circle_x, circle_z, 'k')
ax.set_aspect('equal', adjustable='box')

# Tegn probepositionerne
for i in range(Probe.TOTAL_PROBES):
    ax.scatter(probe[i].position['r'], probe[i].position['z'], marker='o', edgecolors='k', facecolors='w' if probe[i].active else 'C3')
    ax.text(probe[i].position['r'], probe[i].position['z'] - 10, str(probe[i].number), color='black', fontsize=8, ha='center', va='center')

# Navne på akserne og figurtitel
ax.set_xlabel('r [mm]')
ax.set_ylabel('z [mm]')
ax.set_title('Probepositioner i NORTH')

# Figuren gemmes i mappen 'Figures'
fig.savefig(f'{figure_path}/shot_{shot}__probe_positions.png', dpi=300)

Vi vil også indlæse yderligere data fra eksperimentet, fx det målte gastryk eller strømstyrken i de toroidale feltspoler. Vi kalder dette for **maskindata** for at adskille det fra **probedata**, som har med Langmuir-proberne at gøre.

Du får vist en oversigt over alt, der er blevet målt, når du har kørt kodeblokken.

In [ ]:
# Indlæs datafilen, hvori maskindata er gemt
machine_file = nptdms.TdmsFile.read(f"{data_path}/CRIO{shot}.tdms")

# Vis hvilke størrelser, der er blevet gemt som maskindata
for group in machine_file.groups():
    for channel in group.channels():
        print(f"{channel.name}")

## Analysér data

Nu skal vi visualisere og analysere både probedata og maskindata. Vi starter med maskindata.

### Maskindata

Vi vil gerne se, hvordan forskellige størrelser udvikler sig i tid henover skuddet.

Vælg det tidsinterval, som du ønsker at analysere.

In [ ]:
# Disse tal definerer start- og sluttidspunktet (i ms) af det tidsinterval, vi gerne vil plotte i
t_start = 0
t_end = -1

# Her definerer tidsintervallet, som kaldes for 'mask' (dette skal I bare lade være)
t_end = machine_file['Data']['Time'][-1] if t_end == -1 else t_end
mask = (machine_file['Data']['Time'][:] >= t_start) & (machine_file['Data']['Time'][:] <= t_end)

Hvis du ønsker at se hele dataserien, sæt:

`t_start, t_end = 0, -1`

Vi vil nu plotte individuelle størrelser. Alle figurer gemmes i mappen 'Figures'.

In [ ]:
# Plot strømstyrken i de toroidale feltspoler målt i A som funktion af tid målt i ms

fig,ax = plt.subplots()

ax.grid()
ax.plot(machine_file['Data']['Time'][mask], machine_file['Data']['I_TF'][mask], color='purple', linewidth=2.5)
ax.set_xlabel("$t$ [ms]", fontsize=16)
ax.set_ylabel("$I_{\\rm spoler}$ [A]", fontsize=16)
ax.set_title(f"Shot {shot}", fontsize=20)

fig.savefig(f'{figure_path}/shot_{shot}__time_{int(t_start)}_{int(t_end)}_ms__toroidal_field_coil_current.png', dpi=300)

In [ ]:
# Plot gastrykket målt i Pa som funktion af tid målt i ms

fig,ax = plt.subplots()

ax.grid()
ax.plot(machine_file['Data']['Time'][mask], machine_file['Data']['Pressure'][mask]*100, color='orange', linewidth=2.5)
ax.set_xlabel("$t$ [ms]", fontsize=16)
ax.set_ylabel("$p_{\\rm gas}$ [Pa]", fontsize=16)
ax.set_title(f"Shot {shot}", fontsize=20)

fig.savefig(f'{figure_path}/shot_{shot}__time_{int(t_start)}_{int(t_end)}_ms__gas_pressure.png', dpi=300)

In [ ]:
# Plot effekten (energi pr. tid) af mikrobølgekilderne målt i W som funktion af tid målt i ms
# Mikrobølgekilderne bruges til at opvarme plasmaet i NORTH
# Der er både en kilde på indersiden (høj-felt-siden, HFS) og en på ydersiden (lav-felt-siden, LFS) af NORTH

fig,ax = plt.subplots()

ax.grid()
ax.plot(machine_file['Data']['Time'][mask], machine_file['Data']['HFSset'][mask]*450/3000, color='r', linewidth=2.5, label='HFS')
ax.plot(machine_file['Data']['Time'][mask], machine_file['Data']['LFSset'][mask]*450/3000, color='b', linewidth=2.5, label='LFS')
ax.legend(fancybox=False,fontsize=14,loc='upper right')
ax.set_xlabel("$t$ [ms]", fontsize=16)
ax.set_ylabel("$P_{\\rm mikrobølgekilde}$ [W]", fontsize=16)
ax.set_title(f"Shot {shot}", fontsize=20)

fig.savefig(f'{figure_path}/shot_{shot}__time_{int(t_start)}_{int(t_end)}_ms__LFS_microwave_heating.png', dpi=300)

In [ ]:
# Plot signalet fra en lysdiode på NORTH som funktion af tid målt i ms
# Lysdioden måler "mængden" af synligt lys, som kommer ud af NORTH
# Signalet bliver egentlig målt som en spænding, men du kan bare tage det som at signalet er enhedsløst (det vigtige er hvordan signalet ser ud!)
# Plasmaet i NORTH udsender synligt lys, og derfor kan lysdioden hjælpe med at afklare, hvornår der er plasma i NORTH (her burde signalet fra dioden være stort)

fig,ax = plt.subplots()

ax.grid()
ax.plot(machine_file['Data']['Time'][mask], machine_file['Data']['Light'][mask], color='g', linewidth=2.5)
ax.set_xlabel("$t$ [ms]", fontsize=16)
ax.set_ylabel("Lysdiode-signal", fontsize=16)
ax.set_title(f"Shot {shot}", fontsize=20)

fig.savefig(f'{figure_path}/shot_{shot}__time_{int(t_start)}_{int(t_end)}_ms__gas_pressure.png', dpi=300)

Biblioteket 'Matplotlib', som vi bruger til at plotte med, har et væld af muligheder for at ændre udseendet på din graf. Der er masser af hjælp tilgængeligt på Google, hvis du vil ændre udseendet af dit plot.

### Probedata

Det næste trin er at plotte tidsserien for bias-spændingen og strømstyrken i en probe og finde et tidsinterval med brugbar data. Det gøres manuelt ved at kigge på dataen. Prøv dig frem.

Vælg den probe du ønsker at analysere data for, fx for probe 30 sættes `p = 30`

In [ ]:
# Vælg probe
p = 30

Vælg det tidsinterval, som du ønsker at analysere.

In [ ]:
# Disse tal definerer start- og sluttidspunktet (i ms) af det tidsinterval, vi gerne vil plotte i
t_start = 0
t_end = -1

# Dette skal I bare lade være
t_end = probe[p-1].time[-1]*1e3 if t_end == -1 else t_end

Igen, hvis du ønsker at se hele dataserien, sæt:

`t_start, t_end = 0, -1`

Vi vil nu plotte bias-spændingen og strømstyrken i den valgte probe som funktion af tid i det valgte tidsinterval. Alle figurer gemmes i mappen 'Figures'.

In [ ]:
# Bias-spænding målt i V og strømstyrke målt i mA i den valgte probe som funktion af tid målt i ms

idx_start, idx_end = probe[p-1].get_time_indices(t_start*1e-3, t_end*1e-3)

fig, ax = plt.subplots(2, sharex=True)

ax[0].grid()
ax[1].grid()
ax[0].plot(probe[p-1].time[idx_start:idx_end]*1e3, probe[p-1].bias_voltage[idx_start:idx_end], color='k', linewidth=2.5)
ax[1].plot(probe[p-1].time[idx_start:idx_end]*1e3, probe[p-1].current[idx_start:idx_end]*1e3, color='k', linewidth=2.5)
ax[1].set_xlim(t_start, t_end)
ax[0].set_ylabel("$V_{\\rm bias}$ [V]", fontsize=16)
ax[1].set_ylabel("$I_{\\rm probe}$ [mA]", fontsize=16)
ax[1].set_xlabel("$t$ [ms]", fontsize=16)

fig.suptitle(f"Shot {probe[p-1].shot} : Probe {probe[p-1].number}", fontsize=20)
fig.tight_layout()

fig.savefig(f'{figure_path}/shot_{shot}__time_{int(t_start)}_{int(t_end)}_ms__probe_signal.png', dpi=300)

Når du har valgt et passende tidsinterval, kan du hurtigt lave et I-V-plot.

In [ ]:
# Plot I-V-kurven

fig, ax = plt.subplots()

ax.grid()
ax.scatter(probe[p-1].bias_voltage[idx_start:idx_end], probe[p-1].current[idx_start:idx_end]*1e3, 3, color='k')
ax.set_xlabel("$V_{\\rm bias}$ [V]", fontsize=16)
ax.set_ylabel("$I_{\\rm probe}$ [mA]", fontsize=16)
ax.set_title(f"Shot {probe[p-1].shot} : Probe {probe[p-1].number}", fontsize=20)

Nu kan du lave et fit / regression af dataen for at bestemme elektrontemperaturen T_e og elektrontætheden n_e. Tilganen her er at fitte efter ligning (1) i øvelsesvejledningen.

Først defineres den funktion (ligning (1) i øvelsesvejledningen), vi fitter efter.

In [ ]:
def I(V, I_i_sat, V_f, k_BT_e):
    return I_i_sat * (1 - np.exp(1.602e-19*(V-V_f)/k_BT_e))

Så laves fittet ved brug af biblioteket 'scipy'.

In [ ]:
# Gæt på parametre (denne behøves ikke ændres på)
guess = [-0.1, -10, 1e-18]

# Udfør selve regressionen / fittet
popt, pcov = curve_fit(I, probe[p-1].bias_voltage[idx_start:idx_end], probe[p-1].current[idx_start:idx_end]*1e3, guess)
I_i_sat, V_f, k_BT_e = popt

Nu kan vi plotte fittet sammen med data.

In [ ]:
# Plot I-V-kurven + fit

fig, ax = plt.subplots()

ax.grid()
ax.scatter(probe[p-1].bias_voltage[idx_start:idx_end], probe[p-1].current[idx_start:idx_end]*1e3, 3, color='k', label='Data')
ax.plot(probe[p-1].bias_voltage[idx_start:idx_end], I(probe[p-1].bias_voltage[idx_start:idx_end], *popt), color='r', linewidth=2.5, label='Fit')
ax.set_xlabel("$V_{\\rm bias}$ [V]", fontsize=16)
ax.set_ylabel("$I_{\\rm probe}$ [mA]", fontsize=16)
ax.set_title(f"Shot {probe[p-1].shot} : Probe {probe[p-1].number}", fontsize=20)
ax.legend(fancybox=False,fontsize=14)

fig.savefig(f'{figure_path}/shot_{shot}__time_{int(t_start)}_{int(t_end)}_ms__IV_characteristic.png', dpi=300)

print(f"I_i_sat = {I_i_sat:.2f} mA")
print(f"V_f = {V_f:.2f} V")
print(f"k_B*T_e = {k_BT_e/1.602e-19:.2f} eV")

Elektrontemperaturen T_e og elektrontætheden n_e kan nu bestemmes ud fra fitting-parametrene, som du direkte kan aflæse over figuren.

I vores øvelsesvejledning giver vi flere råd til databehandling – der er meget mere at tage fat på, hvis du har tid!